# Training and Evaluation Pipeline

## Training

### set up imports and select options

In [1]:
#%%
from torch.functional import split
from src.datatools import *
from src.engine import train_single_epoch, validate
import torch, torchvision

from torch.utils.data import DataLoader
import torch.nn as nn
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import time
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.optim import lr_scheduler
import os


   
sav_dir='./DCN/'
if not os.path.exists(sav_dir):
    os.mkdir(sav_dir)
RGB_Data_Dir   = '/data/pvraja/greenhouse-data/RGBImages/'
Depth_Data_Dir = '/data/pvraja/greenhouse-data/DepthImages/'  
JSON_Files_Dir = '/data/pvraja/greenhouse-data/GroundTruth/GroundTruth_All_388_Images.json'
split_seed=12    


num_epochs    =  400
ConvType      = 'deformable' # 'standard'
training_category = 'MIMO' #'MIMO', 'MISO', 'SIMO', 'SISO'


if training_category   == 'MIMO':
    transform_type = get_transforms(train=False) 
    inputs=['RGB-D']
    outputs=['All']
    NumOutputs    = 5
    
elif training_category == 'MISO':
    transform_type = get_transforms(train=False)
    inputs=['RGB-D']
    outputs=['FW','DW','H','D','LA']
    NumOutputs    = 1
    
elif training_category == 'SIMO':
    transform_type = get_RGB_transforms(train=False)
    inputs=['RGB','D']
    outputs=['All']
    NumOutputs    = 5
    
elif training_category == 'SISO':
    transform_type = get_RGB_transforms(train=False)
    
    inputs =['RGB','D']
    outputs=['FW','DW','H','D','LA']
    NumOutputs    = 1
    
    
                            
                                


### setup dataset

In [2]:
dataset = GreenhouseDataset(rgb_dir=RGB_Data_Dir, d_dir=Depth_Data_Dir, jsonfile_dir=JSON_Files_Dir, transforms=transform_type) 
dataset.df= dataset.df.iloc[:-50]



train_split, val_split = train_test_split(dataset.df, test_size=0.2, random_state=split_seed, stratify=dataset.df['Variety'])

train = torch.utils.data.Subset(dataset, train_split.index.tolist())
val   = torch.utils.data.Subset(dataset, val_split.index.tolist())
dataset.set_indices(train.indices, val.indices)
                                                                                     
                                
train_loader = torch.utils.data.DataLoader(train, batch_size=10, num_workers=12, shuffle=True)#, sampler=train_sampler)
val_loader   = torch.utils.data.DataLoader(val,   batch_size=10, shuffle=False, num_workers=12)#, sampler=val_sampler)


### define a loss function

In [3]:
from src.nmse import NMSELoss

criterion=NMSELoss()

### Train loop

In [4]:
from src.architecture import GreenhouseMidFusionRegressor
for In in inputs:
    for Out in outputs:
        dataset.input=In
        dataset.out  =Out
                                

        device=torch.device('cuda')
        model= GreenhouseMidFusionRegressor(input_data_type=In, num_outputs=NumOutputs, conv_type=ConvType)
        model.to(device)
        params = [p for p in model.parameters() if p.requires_grad]
        optimizer=torch.optim.Adam(params, lr=0.0005, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)  # select an optimzer for each run
    
                                
        best_val_loss=9999999 # initial dummy value
        current_val_loss=0
        # training_val_loss=0
           
        writer = SummaryWriter()
        start=time.time()
                                
        for epoch in range(num_epochs):
            with open('run.txt', 'a') as f:
                f.write('\n')
                f.write('Epoch: '+ str(epoch+1)+ ', Time Elapsed: '+ str((time.time()-start)/60)+' mins')
            print('Epoch: ', str(epoch+1), ', Time Elapsed: ', str((time.time()-start)/60),' mins')

            train_single_epoch(model, dataset, device, criterion, optimizer, writer, epoch, train_loader)

            best_val_loss=validate(model, dataset, device, training_category, sav_dir, criterion, writer, epoch, val_loader, best_val_loss)

Epoch:  1 , Time Elapsed:  3.64383061726888e-06  mins


/home/pvraja/miniconda3/envs/greenhouse/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train NMSE:  4.976996421813965
Train NMSE:  4.39356803894043
Train NMSE:  3.567845344543457
Train NMSE:  3.1471967697143555
Train NMSE:  3.548143148422241
Train NMSE:  2.867136001586914
Train NMSE:  2.946542978286743
Train NMSE:  2.841763734817505
Train NMSE:  2.6925511360168457
Train NMSE:  2.6476752758026123
Train NMSE:  2.6265029907226562
Train NMSE:  2.3830325603485107
Train NMSE:  2.1940088272094727
Train NMSE:  4.731308937072754
Train NMSE:  2.277210235595703
Train NMSE:  2.3613433837890625
Train NMSE:  2.370082378387451
Train NMSE:  2.3692803382873535
Train NMSE:  2.3765292167663574
Train NMSE:  2.298827648162842
Train NMSE:  2.2000510692596436
Train NMSE:  2.3070549964904785
Train NMSE:  2.0705654621124268
Train NMSE:  2.0631308555603027
Train NMSE:  2.0016465187072754
Train NMSE:  2.150080442428589
Train NMSE:  2.033287525177002
val
Best model Saved! Val NMSE:  14.261265993118286
Epoch:  2 , Time Elapsed:  0.7412668228149414  mins
Train NMSE:  2.075782537460327
Train NMSE:  1.

KeyboardInterrupt: 

## Evalutation

### Define Testset

In [8]:
testset = GreenhouseDataset(rgb_dir=RGB_Data_Dir, d_dir=Depth_Data_Dir, jsonfile_dir=JSON_Files_Dir, transforms=transform_type)

testset_size=50  



testset.df=testset.df[-50:]
# tes

test_loader = torch.utils.data.DataLoader(testset, batch_size=50,num_workers=0, shuffle=False)

### Define Loss functions

In [6]:
cri=NMSELoss()
mse=nn.MSELoss()

### Evaluation Loop

In [11]:
device=torch.device('cuda')
with torch.no_grad():
    for In in inputs:
        final=torch.zeros((testset_size,0))
        all_targets=torch.zeros((testset_size,0))
        for Out in outputs:
            print('Input is ', In)
            testset.input=In
            testset.out  =Out                        

            device=torch.device('cuda')
            model= GreenhouseMidFusionRegressor(input_data_type=In, num_outputs=NumOutputs, conv_type=ConvType)
            model.to(device)
            model.load_state_dict(torch.load(sav_dir+'bestmodel'+training_category+'_' + In + '_' + Out + '.pth'))
            model.eval()


            if Out=='All':
                ap=torch.zeros((0,5))
                at=torch.zeros((0,5))
            else:
                ap=torch.zeros((0,1))
                at=torch.zeros((0,1))

            for rgbd, targets in test_loader:
                rgbd=rgbd.to(device)
                targets=targets.to(device)
                preds=model(rgbd)
                # mse_loss=mse(preds, targets)
                # nmse=criterion(preds, targets)
                # nmse, pred=cri(preds, targets)
                ap=torch.cat((ap, preds.detach().cpu()), 0)
                at=torch.cat((at, targets.detach().cpu()), 0)
            if Out=='All':
                print('FW MSE: ', str(mse(ap[:,0],at[:,0]).tolist()))
                print('DW MSE: ', str(mse(ap[:,1],at[:,1]).tolist()))
                print('H MSE: ', str(mse(ap[:,2],at[:,2]).tolist()))
                print('D MSE: ', str(mse(ap[:,3],at[:,3]).tolist()))
                print('LA MSE: ', str(mse(ap[:,4],at[:,4]).tolist()))
            else:
                final=torch.cat((final, ap.detach().cpu()),1)
                all_targets=torch.cat((all_targets, at.detach().cpu()),1)
                print(Out,' MSE: ', str(mse(ap,at).tolist()))
        if Out=='All':
            print('Overall NMSE: ', str(cri(ap,at).tolist()))
        else:
            print('Overall NMSE: ', str(cri(final,all_targets).tolist()))
        
            
                                    
         

Input is  RGB-D
FW MSE:  16857.876953125
DW MSE:  4.854626655578613
H MSE:  3.97654390335083
D MSE:  22.738414764404297
LA MSE:  5795591.0
Overall NMSE:  1.632205843925476
